# 2023 NCAA Division 1 Cross Country National Qualifier Projections
## This notebook demonstrates how to automate the Kolas points process via systematic web scraping and outputs the results of running the Kolas algorithm on the current regional and national rankings.
## (A detailed explanation of the Kolas system published by the Stride Report can be found [here](https://www.thestridereport.com/understanding-kolas).)

In [6]:
# import relevant libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [7]:
# define helper functions

def get_program_url(link):
  "This function consumes a directathletics.com team link and returns the URL to the associated program's TFRRS page."
  headers = {'user-agent': 'Exploring the viability of creating an automated results aggregator for secondary analysis.'}
  r = requests.get(link, headers=headers)
  page = BeautifulSoup(r.text, 'html')
  url = str(page.find("script").string.split('"')[1])
  return url

def format_results(mf_auto_Qs, mf_at_large_Qs, mf_regional_Qs, gender):
  "This function consumes the qualifier dictionaries and formats the results so that one can easily determine the qualifiers and their positions for each region."
  auto_Qs_dict = {auto_Q_program: 'Q' for auto_Q_program in mf_auto_Qs[gender]}
  at_large_Qs_dict = {at_large_Q_program: at_large_Q_position for (at_large_Q_program, at_large_Q_position) in mf_at_large_Qs[gender]}
  all_Qs_dict = auto_Qs_dict | at_large_Qs_dict

  for region, qualifiers in mf_regional_Qs[gender].items():
    print(f'{region}: {[f"{qualifier} ({all_Qs_dict[qualifier]})" for qualifier in qualifiers]}')

In [36]:
week = 4

# read program information from csv file on my github
# region_df = pd.read_csv("https://raw.githubusercontent.com/Griffin-McCauley/Griffin-McCauley.github.io/main/resources/miscellaneous/regions.csv")
# state_df = pd.read_csv("https://raw.githubusercontent.com/Griffin-McCauley/Griffin-McCauley.github.io/main/resources/miscellaneous/states.csv")
# program_info = pd.merge(region_df, state_df, on="State")[["Program", "Region", "Abbreviation"]]
region_df = pd.read_csv("https://raw.githubusercontent.com/Griffin-McCauley/Griffin-McCauley.github.io/main/resources/miscellaneous/regions.csv")
program_info = region_df[["Program", "Region"]]

ranking_methods = ["regional", "national"]
genders = ['mens', 'womens']

headers = {'user-agent': 'Exploring the viability of creating an automated results aggregator for secondary analysis.'}

for j in range(len(ranking_methods)):
  ranking_method = ranking_methods[j]

  # initialize the dictionaries which will store the results of the Kolas points qualifying process
  mf_auto_Qs = {}
  mf_regional_Qs = {}
  mf_regional_rankings = {}
  mf_full_regionally_ranked_programs_info_dict = {}
  mf_full_nationally_ranked_programs_info_dict = {}
  mf_meet_urls = {}
  mf_program_meets = {}
  mf_beaten_by = {}
  mf_program_kolas_points = {}
  mf_at_large_Qs = {}

  for i in range(len(genders)):

    # add regional substitution functionality
    modified_region = "" #['Great Lakes','Mid-Atlantic','Midwest','Mountain','Northeast','South','South Central','Southeast','West']
    modified_regional_rankings = []

    # scrape regional rankings from https://www.ustfccca.org/
    regional_url = f"https://www.ustfccca.org/2023/10/featured/2023-ncaa-di-{genders[i]}-cross-country-regional-rankings-week-{week}"
    regional_rankings_r = requests.get(regional_url, headers=headers)
    regional_rankings_page = BeautifulSoup(regional_rankings_r.text, 'html')
    region_names = [" ".join(h.string.split(" ")[:-2]) for h in regional_rankings_page.find_all('h6')] # accommodating a small change in the website's html formatting
    # region_names = [h.string for h in regional_rankings_page.find_all('h6')] # accommodating a small change in the website's html formatting
    regional_rankings_tables = regional_rankings_page.find_all("table", "ui small compact celled unstackable definition table")
    regionally_ranked_programs_links_tables = regional_rankings_page.find_all("div", "container-table")
    full_regionally_ranked_programs_info_tuples = []
    top_regional_programs = []
    for regional_table_index, regional_table in enumerate(regional_rankings_tables):
      programs = [a.string for a in regional_table.find_all("a")]
      top_regional_programs.extend(programs[:5])
      regionally_ranked_programs_links = [a['href'] for a in regionally_ranked_programs_links_tables[regional_table_index].find_all("a") if a['href'].startswith("https://www.directathletics.com/teams/xc/")]
      regionally_ranked_programs_urls = [get_program_url(link) for link in regionally_ranked_programs_links]
      regions = [region_names[regional_table_index]]*len(programs)
      full_regionally_ranked_programs_info_tuples.extend(list(zip(programs, regionally_ranked_programs_urls, regions)))
      if region_names[regional_table_index] == modified_region:
        programs[:len(modified_regional_rankings)] = modified_regional_rankings

    full_regionally_ranked_programs_info_dict = {}
    for triplet in full_regionally_ranked_programs_info_tuples:
      full_regionally_ranked_programs_info_dict[triplet[0]] = {"URL": triplet[1], "Region": triplet[2]}


    # scrape national rankings from https://www.ustfccca.org/
    national_url = f"https://www.ustfccca.org/2023/10/featured/2023-ncaa-di-{genders[i]}-cross-country-national-coaches-poll-week-{week}"
    national_rankings_r = requests.get(national_url, headers=headers)
    national_rankings_page = BeautifulSoup(national_rankings_r.text, 'html')
    national_rankings = national_rankings_page.find_all("div", "container-table")
    nationally_ranked_programs = [a.string for a in national_rankings[0].find_all("a") if a['href'].startswith("/polls")]
    nationally_ranked_programs_urls = [full_regionally_ranked_programs_info_dict[nationally_ranked_program]["URL"] for nationally_ranked_program in nationally_ranked_programs]
    nationally_ranked_programs_rankings = [a+1 for a in range(len(nationally_ranked_programs))]
    national_rankings_dict = {"Program": nationally_ranked_programs, "URL": nationally_ranked_programs_urls, "Ranking": nationally_ranked_programs_rankings} #####
    national_rankings_df = pd.DataFrame(national_rankings_dict)
    national_rankings_df = pd.merge(national_rankings_df, program_info, on="Program")
    national_rankings_df = national_rankings_df[["Program", "URL", "Ranking", "Region"]]

    full_nationally_ranked_programs_info_dict = {}
    for row in range(len(national_rankings_df)):
      full_nationally_ranked_programs_info_dict[national_rankings_df.loc[row, "Program"]] = {"URL": national_rankings_df.loc[row, "URL"],
                                                                          "Ranking": national_rankings_df.loc[row, "Ranking"],
                                                                          "Region": national_rankings_df.loc[row, "Region"]}

    top_national_programs = top_regional_programs
    top_national_programs.extend(list(full_nationally_ranked_programs_info_dict.keys()))
    top_national_programs = set(top_national_programs)

    auto_Qs = []
    regional_Qs = {}
    regional_rankings = {}
    # create regionals finishing order from regional rankings or national rankings
    if ranking_method == "regional":
      for regional_program_name, regional_program_info in full_regionally_ranked_programs_info_dict.items():
        if regional_program_info['Region'] not in regional_rankings:
          regional_rankings[regional_program_info['Region']] = [regional_program_name]
        else:
          regional_rankings[regional_program_info['Region']].append(regional_program_name)
    elif ranking_method == "national":
      for national_program_name, national_program_info in full_nationally_ranked_programs_info_dict.items():
        if national_program_info['Region'] not in regional_rankings:
          regional_rankings[national_program_info['Region']] = [national_program_name]
        else:
          regional_rankings[national_program_info['Region']].append(national_program_name)
    for region_name in region_names:
      auto_Qs.extend(regional_rankings[region_name][:2])
      regional_Qs[region_name] = regional_rankings[region_name][:2]


    # scrape relevant meets from https://www.tfrrs.org/
    meet_urls = {}
    program_meets = {}
    for program_name in top_national_programs:
      program_url = full_regionally_ranked_programs_info_dict[program_name]["URL"]
      program_page_r = requests.get(program_url, headers=headers)
      program_page = BeautifulSoup(program_page_r.text, 'html')
      program_results = program_page.find_all("div", "col-lg-8")[1].find_all("td")
      result_index = 0
      while True:
        if result_index % 2 == 0 and not program_results[result_index].string.startswith(("November", "October", "September", "August")):
          break
        elif result_index % 2 == 0 and program_results[result_index].string.startswith(("November", "October", "September", "August")):
          pass
        else:
          if f'https://www.tfrrs.org{program_results[result_index].find("a")["href"]}' not in meet_urls:
            meet_urls[f'https://www.tfrrs.org{program_results[result_index].find("a")["href"]}'] = {"Attendance": 1, "Attendees": [program_name]}
          else:
            meet_urls[f'https://www.tfrrs.org{program_results[result_index].find("a")["href"]}']['Attendance'] += 1
            meet_urls[f'https://www.tfrrs.org{program_results[result_index].find("a")["href"]}']['Attendees'].append(program_name)
          if program_name not in program_meets:
            program_meets[program_name] = [f'https://www.tfrrs.org{program_results[result_index].find("a")["href"]}']
          else:
            program_meets[program_name].append(f'https://www.tfrrs.org{program_results[result_index].find("a")["href"]}')
        result_index += 1


    # scrape auto qualifiers' meet results from https://www.tfrrs.org/
    beaten_by = {}
    program_kolas_points = {}
    for program in auto_Qs:
      program_schedule = program_meets[program]
      for meet in program_schedule:
        if meet_urls[meet]['Attendance'] >= 4: # bumped to 4 in order to mitigate the risk of small, early season meets confounding the Kolas points allocation
          meet_results_r = requests.get(meet, headers=headers)
          meet_results_page = BeautifulSoup(meet_results_r.text, 'html')
          event_titles = meet_results_page.find_all("h3", "font-weight-500")
          event_names = [event.text.split("\n")[0] for event in event_titles]
          if genders[i] == 'mens':
            event_result_indices = [event_index for event_index, event_name in enumerate(event_names) if "Men" in event_name and "Team Results" in event_name]
          elif genders[i] == 'womens':
            event_result_indices = [event_index for event_index, event_name in enumerate(event_names) if "Women" in event_name and "Team Results" in event_name]
          event_results_tables = meet_results_page.find_all("tbody", "color-xc")

          ##### CBU and Colorado St. versus California Baptist and Colorado State on TFRRS and USTFCCCA
          competed_indicator = []
          event_time = []
          for event_result_index in event_result_indices:
            competing_programs = [a.string for a in event_results_tables[event_result_index].find_all("a")]
            if program == 'California Baptist':
              if 'CBU' in competing_programs:
                competed_indicator.append(True)
              else:
                competed_indicator.append(False)
            elif program == 'Colorado State':
              if 'Colorado St.' in competing_programs:
                competed_indicator.append(True)
              else:
                competed_indicator.append(False)
            else:
              if program in competing_programs:
                competed_indicator.append(True)
              else:
                competed_indicator.append(False)
            for td in event_results_tables[event_result_index].find_all("td"):
              if td.string:
                if len(td.string.split(':')) == 2:
                  event_time.append(int(td.string.split(':')[0])*60 + int(td.string.split(':')[1]))
                  break

          for index, event_result_index in enumerate(event_result_indices):
            if competed_indicator[index] and event_time[index] == min(event_time):
              competing_programs = [a.string for a in event_results_tables[event_result_index].find_all("a")]
              for competing_program in competing_programs:
                if program == 'California Baptist':
                  if competing_program != 'CBU':
                    if program not in beaten_by:
                      if competing_program == 'Colorado St.':
                        beaten_by[program] = set(['Colorado State'])
                      else:
                        beaten_by[program] = set([competing_program])
                    else:
                      if competing_program == 'Colorado St.':
                        beaten_by[program].add('Colorado State')
                      else:
                        beaten_by[program].add(competing_program)
                    if competing_program == 'Colorado St.':
                      if 'Colorado State' not in program_kolas_points:
                        program_kolas_points['Colorado State'] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                      else:
                        program_kolas_points['Colorado State']["Defeated Programs"].add(program)
                        program_kolas_points['Colorado State']["Kolas Points"] = len(program_kolas_points['Colorado State']["Defeated Programs"])
                    else:
                      if competing_program not in program_kolas_points:
                        program_kolas_points[competing_program] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                      else:
                        program_kolas_points[competing_program]["Defeated Programs"].add(program)
                        program_kolas_points[competing_program]["Kolas Points"] = len(program_kolas_points[competing_program]["Defeated Programs"])
                  else:
                    break
                elif program == 'Colorado State':
                  if competing_program != 'Colorado St.':
                    if program not in beaten_by:
                      if competing_program == 'CBU':
                        beaten_by[program] = set(['California Baptist'])
                      else:
                        beaten_by[program] = set([competing_program])
                    else:
                      if competing_program == 'CBU':
                        beaten_by[program].add('California Baptist')
                      else:
                        beaten_by[program].add(competing_program)
                    if competing_program == 'CBU':
                      if 'California Baptist' not in program_kolas_points:
                        program_kolas_points['California Baptist'] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                      else:
                        program_kolas_points['California Baptist']["Defeated Programs"].add(program)
                        program_kolas_points['California Baptist']["Kolas Points"] = len(program_kolas_points['California Baptist']["Defeated Programs"])
                    else:
                      if competing_program not in program_kolas_points:
                        program_kolas_points[competing_program] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                      else:
                        program_kolas_points[competing_program]["Defeated Programs"].add(program)
                        program_kolas_points[competing_program]["Kolas Points"] = len(program_kolas_points[competing_program]["Defeated Programs"])
                  else:
                    break
                else:
                  if competing_program != program:
                    if competing_program == "CBU":
                      if program not in beaten_by:
                        beaten_by[program] = set(["California Baptist"])
                      else:
                        beaten_by[program].add("California Baptist")
                      if "California Baptist" not in program_kolas_points:
                        program_kolas_points["California Baptist"] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                      else:
                        program_kolas_points["California Baptist"]["Defeated Programs"].add(program)
                        program_kolas_points["California Baptist"]["Kolas Points"] = len(program_kolas_points["California Baptist"]["Defeated Programs"])
                    elif competing_program == "Colorado St.":
                      if program not in beaten_by:
                        beaten_by[program] = set(["Colorado State"])
                      else:
                        beaten_by[program].add("Colorado State")
                      if "Colorado State" not in program_kolas_points:
                        program_kolas_points["Colorado State"] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                      else:
                        program_kolas_points["Colorado State"]["Defeated Programs"].add(program)
                        program_kolas_points["Colorado State"]["Kolas Points"] = len(program_kolas_points["Colorado State"]["Defeated Programs"])
                    else:
                      if program not in beaten_by:
                        beaten_by[program] = set([competing_program])
                      else:
                        beaten_by[program].add(competing_program)
                      if competing_program not in program_kolas_points:
                        program_kolas_points[competing_program] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                      else:
                        program_kolas_points[competing_program]["Defeated Programs"].add(program)
                        program_kolas_points[competing_program]["Kolas Points"] = len(program_kolas_points[competing_program]["Defeated Programs"])
                  else:
                    break
              break
            else:
              pass
        else:
          pass
          #####


    # select at large bids via the Kolas algorithm
    total_Qs_allowed = 31
    program_kolas_points_tuples = [(program, kolas_dict['Kolas Points']) for program, kolas_dict in program_kolas_points.items()]
    program_kolas_points_tuples.sort(key=lambda x: x[1], reverse=True)
    at_large_Qs = []
    while len(auto_Qs) + len(at_large_Qs) < 32: # includes first one out
      for kolas_tuple_index, kolas_tuple in enumerate(program_kolas_points_tuples):
        if kolas_tuple[0] not in auto_Qs and kolas_tuple[0] not in [at_large_program for (at_large_program, at_large_Q_position) in at_large_Qs]:
          program = kolas_tuple[0]
          push = []
          for regionally_ranked_program in regional_rankings[full_regionally_ranked_programs_info_dict[program]['Region']]:
            if regionally_ranked_program not in auto_Qs and regionally_ranked_program not in [at_large_program for (at_large_program, at_large_qualifying_position) in at_large_Qs] and regionally_ranked_program != program:
              push.append(regionally_ranked_program)
            elif regionally_ranked_program == program:
              break
          if len(push) > 1 or ((len(auto_Qs) + len(at_large_Qs)) < total_Qs_allowed and (len(auto_Qs) + len(at_large_Qs) + 1 + len(push)) > total_Qs_allowed):
            index_shift = 1
            while True:
              if program_kolas_points_tuples[kolas_tuple_index + index_shift][0] not in auto_Qs and program_kolas_points_tuples[kolas_tuple_index + index_shift][0] not in [at_large_program for (at_large_program, at_large_qualifying_position) in at_large_Qs]:
                program = program_kolas_points_tuples[kolas_tuple_index + index_shift][0]
                break
              else:
                index_shift += 1
            push = []
            for regionally_ranked_program in regional_rankings[full_regionally_ranked_programs_info_dict[program]['Region']]:
              if regionally_ranked_program not in auto_Qs and regionally_ranked_program not in [at_large_program for (at_large_program, at_large_qualifying_position) in at_large_Qs] and regionally_ranked_program != program:
                push.append(regionally_ranked_program)
              elif regionally_ranked_program == program:
                break
            if len(push) > 1 or ((len(auto_Qs) + len(at_large_Qs)) < total_Qs_allowed and (len(auto_Qs) + len(at_large_Qs) + 1 + len(push)) > total_Qs_allowed):
              while True:
                if program_kolas_points_tuples[kolas_tuple_index + index_shift][0] not in auto_Qs and program_kolas_points_tuples[kolas_tuple_index + index_shift][0] not in [at_large_program for (at_large_program, at_large_qualifying_position) in at_large_Qs]:
                  program = program_kolas_points_tuples[kolas_tuple_index + index_shift][0]
                  break
                else:
                  index_shift += 1
              push = []
              for regionally_ranked_program in regional_rankings[full_regionally_ranked_programs_info_dict[program]['Region']]:
                if regionally_ranked_program not in auto_Qs and regionally_ranked_program not in [at_large_program for (at_large_program, at_large_qualifying_position) in at_large_Qs] and regionally_ranked_program != program:
                  push.append(regionally_ranked_program)
                elif regionally_ranked_program == program:
                  break
              if len(push) > 1 or ((len(auto_Qs) + len(at_large_Qs)) < total_Qs_allowed and (len(auto_Qs) + len(at_large_Qs) + 1 + len(push)) > total_Qs_allowed):
                while True:
                  if program_kolas_points_tuples[kolas_tuple_index + index_shift][0] not in auto_Qs and program_kolas_points_tuples[kolas_tuple_index + index_shift][0] not in [at_large_program for (at_large_program, at_large_qualifying_position) in at_large_Qs]:
                    program = program_kolas_points_tuples[kolas_tuple_index + index_shift][0]
                    break
                  else:
                    index_shift += 1
                push = []
                for regionally_ranked_program in regional_rankings[full_regionally_ranked_programs_info_dict[program]['Region']]:
                  if regionally_ranked_program not in auto_Qs and regionally_ranked_program not in [at_large_program for (at_large_program, at_large_qualifying_position) in at_large_Qs] and regionally_ranked_program != program:
                    push.append(regionally_ranked_program)
                  elif regionally_ranked_program == program:
                    break
              else:
                pass
            else:
              pass
          else:
            pass

          new_at_large_Qs = push
          new_at_large_Qs.append(program)

          for new_at_large_Q_index, program in enumerate(new_at_large_Qs):
            if len(new_at_large_Qs) > 1:
              if new_at_large_Q_index == 0:
                at_large_Qs.append((program, str(len(auto_Qs) + len(at_large_Qs) + 1) + '*'))
              else:
                at_large_Qs.append((program, str(len(auto_Qs) + len(at_large_Qs) + 1)))
            else:
              at_large_Qs.append((program, str(len(auto_Qs) + len(at_large_Qs) + 1)))
            regional_Qs[full_regionally_ranked_programs_info_dict[program]['Region']].append(program)

            #
            if len(new_at_large_Qs) > 1 and new_at_large_Q_index == 0:
              pass
            else:
              program_schedule = program_meets[program]
              for meet in program_schedule:
                if meet_urls[meet]['Attendance'] >= 4: # bumped to 4 in order to mitigate the risk of small, early season meets confounding the Kolas points allocation
                  meet_results_r = requests.get(meet, headers=headers)
                  meet_results_page = BeautifulSoup(meet_results_r.text, 'html')
                  event_titles = meet_results_page.find_all("h3", "font-weight-500")
                  event_names = [event.text.split("\n")[0] for event in event_titles]
                  if genders[i] == 'mens':
                    event_result_indices = [event_index for event_index, event_name in enumerate(event_names) if "Men" in event_name and "Team Results" in event_name]
                  elif genders[i] == 'womens':
                    event_result_indices = [event_index for event_index, event_name in enumerate(event_names) if "Women" in event_name and "Team Results" in event_name]
                  event_results_tables = meet_results_page.find_all("tbody", "color-xc")

                  ##### CBU and Colorado St. versus California Baptist and Colorado State on TFRRS and USTFCCCA
                  competed_indicator = []
                  event_time = []
                  for event_result_index in event_result_indices:
                    competing_programs = [a.string for a in event_results_tables[event_result_index].find_all("a")]
                    if program == 'California Baptist':
                      if 'CBU' in competing_programs:
                        competed_indicator.append(True)
                      else:
                        competed_indicator.append(False)
                    elif program == 'Colorado State':
                      if 'Colorado St.' in competing_programs:
                        competed_indicator.append(True)
                      else:
                        competed_indicator.append(False)
                    else:
                      if program in competing_programs:
                        competed_indicator.append(True)
                      else:
                        competed_indicator.append(False)
                    for td in event_results_tables[event_result_index].find_all("td"):
                      if td.string:
                        if len(td.string.split(':')) == 2:
                          event_time.append(int(td.string.split(':')[0])*60 + int(td.string.split(':')[1]))
                          break

                  for index, event_result_index in enumerate(event_result_indices):
                    if competed_indicator[index] and event_time[index] == min(event_time):
                      competing_programs = [a.string for a in event_results_tables[event_result_index].find_all("a")]
                      for competing_program in competing_programs:
                        if program == 'California Baptist':
                          if competing_program != 'CBU':
                            if program not in beaten_by:
                              if competing_program == 'Colorado St.':
                                beaten_by[program] = set(['Colorado State'])
                              else:
                                beaten_by[program] = set([competing_program])
                            else:
                              if competing_program == 'Colorado St.':
                                beaten_by[program].add('Colorado State')
                              else:
                                beaten_by[program].add(competing_program)
                            if competing_program == 'Colorado St.':
                              if 'Colorado State' not in program_kolas_points:
                                program_kolas_points['Colorado State'] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                              else:
                                program_kolas_points['Colorado State']["Defeated Programs"].add(program)
                                program_kolas_points['Colorado State']["Kolas Points"] = len(program_kolas_points['Colorado State']["Defeated Programs"])
                            else:
                              if competing_program not in program_kolas_points:
                                program_kolas_points[competing_program] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                              else:
                                program_kolas_points[competing_program]["Defeated Programs"].add(program)
                                program_kolas_points[competing_program]["Kolas Points"] = len(program_kolas_points[competing_program]["Defeated Programs"])
                          else:
                            break
                        elif program == 'Colorado State':
                          if competing_program != 'Colorado St.':
                            if program not in beaten_by:
                              if competing_program == 'CBU':
                                beaten_by[program] = set(['California Baptist'])
                              else:
                                beaten_by[program] = set([competing_program])
                            else:
                              if competing_program == 'CBU':
                                beaten_by[program].add('California Baptist')
                              else:
                                beaten_by[program].add(competing_program)
                            if competing_program == 'CBU':
                              if 'California Baptist' not in program_kolas_points:
                                program_kolas_points['California Baptist'] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                              else:
                                program_kolas_points['California Baptist']["Defeated Programs"].add(program)
                                program_kolas_points['California Baptist']["Kolas Points"] = len(program_kolas_points['California Baptist']["Defeated Programs"])
                            else:
                              if competing_program not in program_kolas_points:
                                program_kolas_points[competing_program] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                              else:
                                program_kolas_points[competing_program]["Defeated Programs"].add(program)
                                program_kolas_points[competing_program]["Kolas Points"] = len(program_kolas_points[competing_program]["Defeated Programs"])
                          else:
                            break
                        else:
                          if competing_program != program:
                            if competing_program == "CBU":
                              if program not in beaten_by:
                                beaten_by[program] = set(["California Baptist"])
                              else:
                                beaten_by[program].add("California Baptist")
                              if "California Baptist" not in program_kolas_points:
                                program_kolas_points["California Baptist"] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                              else:
                                program_kolas_points["California Baptist"]["Defeated Programs"].add(program)
                                program_kolas_points["California Baptist"]["Kolas Points"] = len(program_kolas_points["California Baptist"]["Defeated Programs"])
                            elif competing_program == "Colorado St.":
                              if program not in beaten_by:
                                beaten_by[program] = set(["Colorado State"])
                              else:
                                beaten_by[program].add("Colorado State")
                              if "Colorado State" not in program_kolas_points:
                                program_kolas_points["Colorado State"] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                              else:
                                program_kolas_points["Colorado State"]["Defeated Programs"].add(program)
                                program_kolas_points["Colorado State"]["Kolas Points"] = len(program_kolas_points["Colorado State"]["Defeated Programs"])
                            else:
                              if program not in beaten_by:
                                beaten_by[program] = set([competing_program])
                              else:
                                beaten_by[program].add(competing_program)
                              if competing_program not in program_kolas_points:
                                program_kolas_points[competing_program] = {"Kolas Points": 1, "Defeated Programs": set([program])}
                              else:
                                program_kolas_points[competing_program]["Defeated Programs"].add(program)
                                program_kolas_points[competing_program]["Kolas Points"] = len(program_kolas_points[competing_program]["Defeated Programs"])
                          else:
                            break
                      break
                    else:
                      pass
                else:
                  pass
                  #####

              program_kolas_points_tuples = [(kolas_program, kolas_dict['Kolas Points']) for kolas_program, kolas_dict in program_kolas_points.items()]
              program_kolas_points_tuples.sort(key=lambda x: x[1], reverse=True)
            #

          break
        else:
          pass

    if genders[i] == 'mens':
      mf_auto_Qs['Male'] = auto_Qs
      mf_regional_Qs['Male'] = regional_Qs
      mf_regional_rankings['Male'] = regional_rankings
      mf_full_regionally_ranked_programs_info_dict['Male'] = full_regionally_ranked_programs_info_dict
      mf_full_nationally_ranked_programs_info_dict['Male'] = full_nationally_ranked_programs_info_dict
      mf_meet_urls['Male'] = meet_urls
      mf_program_meets['Male'] = program_meets
      mf_beaten_by['Male'] = beaten_by
      mf_program_kolas_points['Male'] = program_kolas_points
      mf_at_large_Qs['Male'] = at_large_Qs
    elif genders[i] == 'womens':
      mf_auto_Qs['Female'] = auto_Qs
      mf_regional_Qs['Female'] = regional_Qs
      mf_regional_rankings['Female'] = regional_rankings
      mf_full_regionally_ranked_programs_info_dict['Female'] = full_regionally_ranked_programs_info_dict
      mf_full_nationally_ranked_programs_info_dict['Female'] = full_nationally_ranked_programs_info_dict
      mf_meet_urls['Female'] = meet_urls
      mf_program_meets['Female'] = program_meets
      mf_beaten_by['Female'] = beaten_by
      mf_program_kolas_points['Female'] = program_kolas_points
      mf_at_large_Qs['Female'] = at_large_Qs


  for idx, gender in enumerate(['Male','Female']):
    print('\n')
    if gender == 'Male':
      if ranking_method == 'regional':
        print("NCAA D1 Men's Regional Qualifiers (Based on Regional Rankings)")
      elif ranking_method == 'national':
        print("NCAA D1 Men's Regional Qualifiers (Based on National Rankings)")
    elif gender == 'Female':
      if ranking_method == 'regional':
        print("NCAA D1 Women's Regional Qualifiers (Based on Regional Rankings)")
      elif ranking_method == 'national':
        print("NCAA D1 Women's Regional Qualifiers (Based on National Rankings)")
    format_results(mf_auto_Qs, mf_at_large_Qs, mf_regional_Qs, gender)
    if idx == 1:
      print('\n')




NCAA D1 Men's Regional Qualifiers (Based on Regional Rankings)
Great Lakes: ['Notre Dame (Q)', 'Michigan (Q)', 'Wisconsin (23*)', 'Butler (24)', 'Michigan State (31)']
Mid-Atlantic: ['Villanova (Q)', 'Princeton (Q)']
Midwest: ['Oklahoma State (Q)', 'Iowa State (Q)']
Mountain: ['Northern Arizona (Q)', 'BYU (Q)', 'Colorado (19)', 'New Mexico (26)', 'Montana State (27*)', 'Air Force (28)']
Northeast: ['Syracuse (Q)', 'Harvard (Q)', 'Iona (32)']
South: ['Tennessee (Q)', 'Alabama (Q)', 'Florida State (29)']
South Central: ['Texas (Q)', 'Arkansas (Q)']
Southeast: ['Furman (Q)', 'North Carolina (Q)', 'Virginia (20*)', 'Wake Forest (21)', 'Eastern Kentucky (22)']
West: ['California Baptist (Q)', 'Boise State (Q)', 'Stanford (25)', 'Gonzaga (30)']


NCAA D1 Women's Regional Qualifiers (Based on Regional Rankings)
Great Lakes: ['Notre Dame (Q)', 'Michigan State (Q)', 'Wisconsin (26)']
Mid-Atlantic: ['Georgetown (Q)', 'Penn State (Q)']
Midwest: ['Oklahoma State (Q)', 'Iowa State (Q)']
Mountain: